In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re

In [2]:
resp = requests.get('https://www.mcdonalds.com/us/en-us/full-menu.html')
soup = BeautifulSoup(resp.content, 'html.parser')

In [3]:
menu_categories = soup.findAll('a', class_ = 'category-link')
menulinks = [f"https://www.mcdonalds.com{item.attrs['href']}" for item in menu_categories][1:]

In [4]:
def get_item_links(menulink):
    resp = requests.get(menulink)
    soup = BeautifulSoup(resp.content, 'html.parser')
    items = soup.findAll('a', class_ = "categories-item-link")
    itemlinks = [f"https://www.mcdonalds.com{item.attrs['href']}" for item in items]
    return itemlinks

In [8]:
def get_item_name_and_cal(itemlink):
    resp = requests.get(itemlink)
    soup = BeautifulSoup(resp.content, 'html.parser')
    try:
        item_name = soup.find('h1', class_ = "heading typo-h1").get_text()
        calories = soup.find('div', class_ = 'calorie-count').get_text()
        calories = re.sub("\D", "", calories)
        calories = int(calories[:len(calories)//2])
        return {item_name : calories}
    except:
        return {'Side Salad' : 15}

In [17]:
cals_df = pd.DataFrame()
for link in menulinks:
    itemlinks = get_item_links(link)
    cals_dict = {}
    for link in itemlinks:
        cals_dict.update(get_item_name_and_cal(link))
    cals_df = cals_df.append(pd.DataFrame.from_dict(cals_dict, orient = 'index'))
cals_df.reset_index(inplace = True)
cals_df.columns = ['Item', 'Calories']
cals_df

,Item,Calories
0,Egg McMuffin®,300
1,Sausage McMuffin®,400
2,Sausage McMuffin® with Egg,480
3,"Bacon, Egg & Cheese Biscuit",450
4,Sausage Biscuit,460
...,...,...
125,Fruit 'N Yogurt Parfait,210
126,Side Salad,15
127,Minute Maid® Blue Raspberry Slushie,190
128,Minute Maid® Fruit Punch Slushie,170


In [18]:
cals_df.describe()

,Calories
count,130.000000
mean,326.500000
std,216.345304
min,0.000000
25%,170.000000
50%,300.000000
75%,470.000000
max,1340.000000


In [27]:
cals_df

,Item,Calories
0,Egg McMuffin®,300
1,Sausage McMuffin®,400
2,Sausage McMuffin® with Egg,480
3,"Bacon, Egg & Cheese Biscuit",450
4,Sausage Biscuit,460
...,...,...
125,Fruit 'N Yogurt Parfait,210
126,Side Salad,15
127,Minute Maid® Blue Raspberry Slushie,190
128,Minute Maid® Fruit Punch Slushie,170


In [31]:
cals_df.loc[cals_df['Calories'].idxmax()]

Item        Big Breakfast® with Hotcakes
Calories                            1340
Name: 45, dtype: object